In [8]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl

today = date.today()
#today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'01.24.2022'

In [ ]:
## DKNG Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

shutil.move(downloads + "/"+ "DKSalaries.csv",
           contest_data_loc + "/"+ "DKSalaries.csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + str(today)+".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries.csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("DKSalaries.csv")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

In [9]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/DraftKings'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("DK NBA Projections.csv")

In [10]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\DraftKings\ETR_Daily_' + str(today) + ".xlsx")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              dkng[['Name','Roster Position', 'Name + ID']], 
              left_on='Player', 
              right_on='Name')

In [11]:
df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID
0,Jonas Valanciunas,NOP,IND,32.5,C,"8,300",46.8,7.6,54.4,27.0,MAIN,Jonas Valanciunas,C/UTIL,Jonas Valanciunas (21106726)
1,Jordan Clarkson,UTA,PHX,34.0,PG/SG,"6,700",39.3,6.5,47.0,42.6,MAIN,Jordan Clarkson,PG/SG/G/UTIL,Jordan Clarkson (21106775)
2,Herbert Jones,NOP,IND,35.5,PF/C,"4,700",30.6,5.8,38.4,43.0,MAIN,Herbert Jones,PF/C/F/UTIL,Herbert Jones (21106849)
3,Hassan Whiteside,UTA,PHX,30.0,C,"6,300",36.5,5.3,44.2,21.6,MAIN,Hassan Whiteside,C/UTIL,Hassan Whiteside (21106791)
4,Jared Butler,UTA,PHX,27.0,PG,"3,000",22.4,4.4,30.7,0.0,MAIN,Jared Butler,PG/G/UTIL,Jared Butler (21107199)


In [12]:
## Unnests Roster Positions into each Row (explode in Python) ##


## Update Salary to INT 
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Roster Position'].astype(str)
df['Roster Position'] = df['Roster Position'].str.split('/')

df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID
0,Jonas Valanciunas,NOP,IND,32.5,C,8300.0,46.8,7.6,54.4,27.0,MAIN,Jonas Valanciunas,"[C, UTIL]",Jonas Valanciunas (21106726)
1,Jordan Clarkson,UTA,PHX,34.0,PG/SG,6700.0,39.3,6.5,47.0,42.6,MAIN,Jordan Clarkson,"[PG, SG, G, UTIL]",Jordan Clarkson (21106775)
2,Herbert Jones,NOP,IND,35.5,PF/C,4700.0,30.6,5.8,38.4,43.0,MAIN,Herbert Jones,"[PF, C, F, UTIL]",Herbert Jones (21106849)
3,Hassan Whiteside,UTA,PHX,30.0,C,6300.0,36.5,5.3,44.2,21.6,MAIN,Hassan Whiteside,"[C, UTIL]",Hassan Whiteside (21106791)
4,Jared Butler,UTA,PHX,27.0,PG,3000.0,22.4,4.4,30.7,0.0,MAIN,Jared Butler,"[PG, G, UTIL]",Jared Butler (21107199)


In [13]:
df.to_excel("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)
os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

0

# Create the Constraint Problem

Goal: Maximize DK Points

- Total Players = 8
- TotalSalary <= 50000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_G
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_F
- TotalPosition_C
- TotalPosition_UTIL

In [30]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

In [31]:
player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [32]:
prob += pulp.lpSum(player_var for player_var in player_vars) == 8

In [33]:
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['Roster Position'].iloc[i]) for i in range(len(df))])
    

# prob += get_position_sum(player_vars, df, 'PG') >= 1
# prob += get_position_sum(player_vars, df, 'SG') >= 1
# prob += get_position_sum(player_vars, df, 'G') >= 1
# prob += get_position_sum(player_vars, df, 'SF') >= 1
# prob += get_position_sum(player_vars, df, 'PF') >= 1
# prob += get_position_sum(player_vars, df, 'F') >= 1
# prob += get_position_sum(player_vars, df, 'C') >= 1
# prob += get_position_sum(player_vars, df, 'UTIL') >= 1

prob += get_position_sum(player_vars, df, 'PG') <= 2
prob += get_position_sum(player_vars, df, 'PG') >= 1
prob += get_position_sum(player_vars, df, 'SG') <= 3
prob += get_position_sum(player_vars, df, 'SG') >= 1
prob += get_position_sum(player_vars, df, 'SF') <= 3
prob += get_position_sum(player_vars, df, 'SF') >= 1
prob += get_position_sum(player_vars, df, 'PF') <= 3
prob += get_position_sum(player_vars, df, 'PF') >= 1
prob += get_position_sum(player_vars, df, 'C') <= 2
prob += get_position_sum(player_vars, df, 'C') >= 1

In [34]:
# total salary constraint

prob += pulp.lpSum(df['DK Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 50000

In [35]:
# finally, specify the goal
prob += pulp.lpSum([df['DK Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [36]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [37]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['Roster Position'], row.Player, row.Team, row['DK Salary'], row['DK Points'])
        total_salary_used += row['DK Salary']
        mean_AvgPointsPerGame += row['DK Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

['C', 'UTIL'] Jonas Valanciunas NOP 8300.0 46.8
['PG', 'SG', 'G', 'UTIL'] Jordan Clarkson UTA 6700.0 39.3
['PG', 'G', 'UTIL'] Jared Butler UTA 3000.0 22.4
['SG', 'G', 'UTIL'] Trent Forrest UTA 3000.0 22.3
['SF', 'PF', 'F', 'UTIL'] Eric Paschall UTA 3300.0 23.0
['C', 'UTIL'] Nikola Vucevic CHI 9500.0 47.7
['SG', 'SF', 'F', 'G', 'UTIL'] Zach LaVine CHI 8400.0 39.8
['SF', 'PF', 'F', 'UTIL'] Josh Hart NOP 7400.0 35.6


(49600.0, 276.90000000000003)

In [ ]:
df.dtypes